# GenParse: Lark Interface

In [1]:
import lark

from genparse.util import LarkStuff
from arsenal import Integerizer
from arsenal.maths import compare
from collections import Counter

## Using lark as a front end

In [2]:
grammar2 = r"""
WS: /[ \t\f\r\n]/
STAR: "*"
NUMBER: /\d+/

start: WS "SELECT" WS select_expr WS "FROM" WS from_expr
  [WS "WHERE" WS bool_condition] [WS "GROUP BY" WS var_list] [WS "ORDER BY" WS orderby_expr] WS EOS
EOS: "</s>"
select_expr: STAR | select_list
bool_condition: bool_expr | "(" bool_condition WS "AND" WS bool_condition ")" | "(" bool_condition WS "OR" WS bool_condition ")" 
bool_expr: var "=" value | var ">" value | var "<" value
from_expr: "data"
orderby_expr: var_list WS "ASC" | var_list WS "DESC"
select_list: select_var ("," WS select_var)*
var_list: var ("," WS var)*
select_var: var | "AVG(" var ")" | "MEDIAN(" var ")" | "COUNT(" var ")"
var: "age" | "gender" | "year" | "state_color" | "zipcode" | "vote" | "race_ethnicity"
value: NUMBER | "red" | "blue" | "white" | "black" | "latino" | "republican" | "democrat" | "male" | "female"

"""

In [3]:
grammar1 = """
start: query_expr EOS

EOS: "</s>"

query_expr: select [ "ORDER" "BY" (order_by_expr ",")*  order_by_expr] [ "LIMIT" integer_ ] 

select: "SELECT" [(select_expr ",")*] select_expr "FROM" "data" [ "WHERE" bool_expression ] [ "GROUP" "BY" [(expression ",")*] expression ]

select_expr.0: expression_math [ [ "AS" ] alias ] -> select_expression

?expression_math: expression_product
               | expression_math PLUS expression_product -> expression_add
               | expression_math "-" expression_product -> expression_sub
               | AGGREGATION expression_math /\)/ -> sql_aggregation

?expression: (name | STAR) -> column_name
            | literal

?expression_product: expression_parens
                  | expression_product STAR expression_parens
                  | expression_product "/" expression_parens 

?expression_parens: expression
                  | /\(/ expression_parens STAR expression /\)/ 
                  | /\(/  expression_parens "/" expression /\)/ 
                  | /\(/  expression_parens PLUS expression /\)/
                  | /\(/  expression_parens "-" expression /\)/

bool_expression: bool_parentheses
                 | bool_expression "AND" bool_parentheses 
                 | bool_expression "OR" bool_parentheses
bool_parentheses: comparison_type
                 | /\(/   bool_expression "AND" comparison_type /\)/
                 | /\(/  bool_expression "OR" comparison_type /\)/
comparison_type: equals | not_equals | greater_than | less_than | greater_than_or_equal
| less_than_or_equal | is_null | is_not_null
equals: expression_math "=" expression_math
not_equals: expression_math ("<>" | "!=") expression_math
greater_than: expression_math ">" expression_math
less_than: expression_math "<" expression_math
greater_than_or_equal: expression_math ">=" expression_math
less_than_or_equal: expression_math "<=" expression_math
is_null: expression_math "is" "null"
is_not_null: expression_math "is" "not" "null"

alias: /[A-Za-z]+/
name: /[A-Za-z]+/
PLUS: /\+/

order_by_expr: expression_math ["ASC"] -> order_asc
        | expression_math "DESC" -> order_desc

AGGREGATION.8: ("sum(" | "avg(" | "min(" | "max(" | "count(" "distinct" | "count(")
STAR: /\*/
integer_: /[1-9][0-9]*/
?literal: boolean -> bool
       | integer_ -> number
       | ESCAPED_STRING -> string

boolean: "true" -> true
       | "false" -> false

%import common.WS
%ignore WS
%import common.ESCAPED_STRING
    
"""

The following code is adapted from partenon.

In [4]:
raw_grammar = grammar1

lark_stuff = LarkStuff(raw_grammar)

In [5]:
intern = Integerizer()   # rename nonterminals to integers
g = lark_stuff.convert()
g = g.rename(intern)
assert g.in_cnf()    # lark returns a grammar in CNF
#g = g.cnf

In [6]:
g

0.5: 1 → 2 3
0.06666666666666667: 4 → 5 6
1.0: 7 → 8 2
0.058823529411764705: 2 → 9 10
1.0: 11 → 12 13
1.0: 14 → ORDER
0.08333333333333333: 9 → 5 15
0.08333333333333333: 16 → 17 18
1.0: 19 → 20 21
1.0: 22 → 8 12
1.0: 23 → 20 24
1.0: 25 → 4 26
0.06666666666666667: 4 → 5 15
1.0: 10 → 27 28
1.0: 29 → 30 23
1.0: 31 → 32 33
0.07692307692307693: 30 → 30 34
0.1111111111111111: 35 → 4 36
0.1: 28 → 5 37
1.0: 38 → 39 40
1.0: 24 → 8 12
0.16666666666666666: 12 → TRUE
1.0: 41 → 35 13
1.0: 42 → 4 36
1.0: 43 → 44 45
0.058823529411764705: 2 → 4 46
1.0: 36 → 47 48
0.07692307692307693: 30 → 4 49
0.058823529411764705: 2 → STAR
0.058823529411764705: 50 → 17 51
0.1: 28 → 5 15
1.0: 52 → 14 53
0.1111111111111111: 35 → 4 54
0.1111111111111111: 35 → 4 55
0.07692307692307693: 30 → 5 56
1.0: 57 → 58 9
0.25: 59 → FALSE
0.058823529411764705: 60 → 4 33
1.0: 61 → EQUAL
1.0: 62 → 12 13
0.1: 28 → ESCAPED_STRING
0.08333333333333333: 9 → 9 10
1.0: 63 → 44 64
1.0: 65 → __ANON_7
0.5: 66 → 4 67
1.0: 68 → 60 69
1.0: 5 → __ANON_1
0.06666666666666667: 4 → 9 10
1.0: 70 → 60 71
0.16666666666666666: 12 → FALSE
1.0: 72 → 61 4
1.0: 73 → 20 74
0.058823529411764705: 2 → 5 75
1.0: 21 → 8 76
0.1111111111111111: 35 → 4 26
0.08333333333333333: 16 → 17 51
0.058823529411764705: 60 → __ANON_7
0.16666666666666666: 12 → __ANON_6
0.058823529411764705: 2 → TRUE
1.0: 54 → 47 77
1.0: 47 → IS
1.0: 78 → 79 65
0.058823529411764705: 50 → 16 80
0.06666666666666667: 4 → 4 46
1.0: 81 → 4 82
1.0: 83 → 84 29
0.07692307692307693: 30 → 4 36
0.058823529411764705: 60 → 85 86
0.08333333333333333: 9 → __ANON_7
0.058823529411764705: 60 → 5 37
1.0: 87 → __ANON_4
1.0: 88 → 8 12
0.06666666666666667: 4 → __ANON_7
1.0: 71 → 39 89
1.0: 90 → 60 91
0.5: 92 → TRUE
0.058823529411764705: 2 → FALSE
1.0: 93 → 30 94
0.058823529411764705: 2 → ESCAPED_STRING
1.0: 95 → 84 93
1.0: 96 → 97 41
0.1: 28 → STAR
0.058823529411764705: 2 → __ANON_6
0.058823529411764705: 60 → 5 15
0.07692307692307693: 30 → 4 54
0.07692307692307693: 30 → 4 55
1.0: 98 → 8 99
0.058823529411764705: 50 → 17 100
0.08333333333333333: 9 → TRUE
1.0: 17 → SELECT
0.09090909090909091: 101 → 4 36
0.06666666666666667: 4 → TRUE
0.08333333333333333: 9 → 5 37
1.0: 102 → 20 103
0.058823529411764705: 2 → 4 104
0.058823529411764705: 50 → 17 105
1.0: 104 → DESC
0.06666666666666667: 4 → 5 37
1.0: 0 → 50 106
0.5: 92 → FALSE
0.07692307692307693: 30 → 30 107
0.1: 28 → 5 75
1.0: 69 → 39 43
1.0: 75 → 28 108
0.09090909090909091: 101 → 4 54
0.09090909090909091: 101 → 4 55
1.0: 109 → 20 110
0.1: 28 → TRUE
1.0: 111 → 44 112
1.0: 113 → 114 115
1.0: 44 → DATA
0.08333333333333333: 16 → 17 100
1.0: 116 → 44 102
1.0: 117 → 118 12
1.0: 119 → NOT
0.08333333333333333: 9 → ESCAPED_STRING
1.0: 120 → 12 3
0.058823529411764705: 60 → 9 10
1.0: 121 → 39 122
0.1111111111111111: 35 → 4 72
0.06666666666666667: 4 → ESCAPED_STRING
1.0: 123 → 60 121
0.5: 118 → 118 120
1.0: 8 → BY
1.0: 18 → 114 124
0.09090909090909091: 101 → 4 26
1.0: 125 → 4 126
0.058823529411764705: 50 → 16 127
1.0: 46 → 128 9
0.058823529411764705: 2 → 4 57
1.0: 107 → 129 101
0.058823529411764705: 50 → 17 123
1.0: 130 → 27 11
1.0: 131 → 84 30
0.058823529411764705: 50 → 17 90
0.1: 28 → FALSE
1.0: 58 → PLUS
1.0: 34 → 97 101
0.058823529411764705: 60 → 4 46
1.0: 76 → 118 12
0.058823529411764705: 60 → STAR
0.1111111111111111: 35 → 4 82
0.1: 28 → __ANON_6
1.0: 26 → 132 4
1.0: 33 → __ANON_6
1.0: 124 → 60 133
0.5: 66 → 4 55
0.1111111111111111: 35 → 4 126
1.0: 99 → 1 134
1.0: 134 → 2 135
1.0: 136 → __ANON_6
1.0: 137 → 79 65
1.0: 138 → 58 62
0.08333333333333333: 16 → 17 123
0.5: 118 → 12 3
0.058823529411764705: 50 → 17 139
0.058823529411764705: 50 → 17 68
0.08333333333333333: 16 → 17 90
0.058823529411764705: 60 → 5 75
0.058823529411764705: 50 → 17 140
1.0: 91 → 39 111
0.058823529411764705: 2 → 9 141
0.08333333333333333: 9 → STAR
1.0: 56 → 30 142
1.0: 143 → 39 116
0.058823529411764705: 50 → 17 144
0.06666666666666667: 4 → STAR
0.058823529411764705: 60 → TRUE
1.0: 105 → 60 145
1.0: 103 → 8 146
0.058823529411764705: 2 → 5 6
1.0: 48 → 119 77


In [7]:
len(g.rules), len(g.V), len(g.N)

(320, 38, 184)

In [8]:
sorted(g.cnf.V)

['AGGREGATION',
 'AND',
 'AS',
 'ASC',
 'BY',
 'COMMA',
 'DATA',
 'DESC',
 'EOS',
 'EQUAL',
 'ESCAPED_STRING',
 'FALSE',
 'FROM',
 'GROUP',
 'IS',
 'LESSTHAN',
 'LIMIT',
 'MINUS',
 'MORETHAN',
 'NOT',
 'NULL',
 'OR',
 'ORDER',
 'PLUS',
 'SELECT',
 'SLASH',
 'STAR',
 'TRUE',
 'WHERE',
 'WS',
 '__ANON_0',
 '__ANON_1',
 '__ANON_2',
 '__ANON_3',
 '__ANON_4',
 '__ANON_5',
 '__ANON_6',
 '__ANON_7']

In [9]:
#from newton.linking import LinkAnalysis
#f = Integerizer()
#links = LinkAnalysis(g.rename(f))
#links.dfs

In [10]:
g.language(6)

Counter({('SELECT', '__ANON_7', 'FROM', 'DATA', 'EOS'): 0.0034602076124567475,
         ('SELECT', 'STAR', 'FROM', 'DATA', 'EOS'): 0.0034602076124567475,
         ('SELECT', 'TRUE', 'FROM', 'DATA', 'EOS'): 0.0034602076124567475,
         ('SELECT', 'FALSE', 'FROM', 'DATA', 'EOS'): 0.0034602076124567475,
         ('SELECT',
          'ESCAPED_STRING',
          'FROM',
          'DATA',
          'EOS'): 0.0034602076124567475,
         ('SELECT', '__ANON_6', 'FROM', 'DATA', 'EOS'): 0.0034602076124567475,
         ('SELECT',
          '__ANON_7',
          'FROM',
          'DATA',
          'LIMIT',
          '__ANON_7',
          'EOS'): 0.0002883506343713956,
         ('SELECT',
          'STAR',
          'FROM',
          'DATA',
          'LIMIT',
          '__ANON_7',
          'EOS'): 0.0002883506343713956,
         ('SELECT',
          'TRUE',
          'FROM',
          'DATA',
          'LIMIT',
          '__ANON_7',
          'EOS'): 0.0002883506343713956,
         ('SELECT',

## Tokenization

We can extract lark's tokenizer in a format that we can build on.  We will even make a DIY tokenizer based on Python's `re` library.

| Terminology  |         |
|--------------|---------|
| tokenization | lexing  |
| tokenizers   | lexers  |
| tokens       | lexemes |



In [11]:
sorted(lark_stuff.all_terminals, key=lambda t: -t.priority)

[TerminalDef('AGGREGATION', '(?:count\\(distinct|count\\(|sum\\(|avg\\(|min\\(|max\\()'),
 TerminalDef('ESCAPED_STRING', '".*?(?<!\\\\)(\\\\\\\\)*?"'),
 TerminalDef('WS', '(?:[ \t\x0c\r\n])+'),
 TerminalDef('EOS', '</s>'),
 TerminalDef('PLUS', '\\+'),
 TerminalDef('STAR', '\\*'),
 TerminalDef('COMMA', ','),
 TerminalDef('ORDER', 'ORDER'),
 TerminalDef('BY', 'BY'),
 TerminalDef('LIMIT', 'LIMIT'),
 TerminalDef('WHERE', 'WHERE'),
 TerminalDef('GROUP', 'GROUP'),
 TerminalDef('SELECT', 'SELECT'),
 TerminalDef('FROM', 'FROM'),
 TerminalDef('DATA', 'data'),
 TerminalDef('AS', 'AS'),
 TerminalDef('MINUS', '\\-'),
 TerminalDef('__ANON_0', '\\)'),
 TerminalDef('SLASH', '/'),
 TerminalDef('__ANON_1', '\\('),
 TerminalDef('AND', 'AND'),
 TerminalDef('OR', 'OR'),
 TerminalDef('EQUAL', '='),
 TerminalDef('__ANON_2', '<>'),
 TerminalDef('__ANON_3', '!='),
 TerminalDef('MORETHAN', '>'),
 TerminalDef('LESSTHAN', '<'),
 TerminalDef('__ANON_4', '>='),
 TerminalDef('__ANON_5', '<='),
 TerminalDef('IS', 'i

### DIY tokenizer

In [12]:
text = "12 + 24 - 36 * 48 / 60 SELECT table.name AS thing WHERE table.potato IS NOT 'banana'"

for x, y in lark_stuff.simple_tokenizer(text):
    print(f'{x:15s} -> {y!r}')

__ANON_7        -> '12'
PLUS            -> '+'
__ANON_7        -> '24'
MINUS           -> '-'
__ANON_7        -> '36'
STAR            -> '*'
__ANON_7        -> '48'
SLASH           -> '/'
__ANON_7        -> '60'
SELECT          -> 'SELECT'
__ANON_6        -> 'table'
__ANON_6        -> 'name'
AS              -> 'AS'
__ANON_6        -> 'thing'
WHERE           -> 'WHERE'
__ANON_6        -> 'table'
__ANON_6        -> 'potato'
__ANON_6        -> 'IS'
__ANON_6        -> 'NOT'
__ANON_6        -> 'banana'


### Parsing tokenized input

In [13]:
text = 'SELECT name FROM data </s>'

In [14]:
tokens = list(lark_stuff.lex(text))
tokens

[Token('SELECT', 'SELECT'),
 Token('__ANON_6', 'name'),
 Token('FROM', 'FROM'),
 Token('DATA', 'data'),
 Token('EOS', '</s>')]

Call the lark parser on the text:

In [15]:
lark_stuff.instance.parse(text)

Tree(Token('RULE', 'start'), [Tree(Token('RULE', 'query_expr'), [Tree(Token('RULE', 'select'), [Tree('select_expression', [Tree('column_name', [Tree(Token('RULE', 'name'), [Token('__ANON_6', 'name')])]), None]), None, None]), None, None]), Token('EOS', '</s>')])

We can call the lark parser on these tokens:

In [16]:
lark_stuff.parser.parse(tokens, 'start')

Tree(NonTerminal(Token('RULE', 'start')), [Tree(NonTerminal(Token('RULE', 'query_expr')), [Tree(NonTerminal(Token('RULE', 'select')), [Token('SELECT', 'SELECT'), Tree(NonTerminal(Token('RULE', 'select_expr')), [Tree(NonTerminal(Token('RULE', 'expression_math')), [Tree(NonTerminal(Token('RULE', 'expression_product')), [Tree(NonTerminal(Token('RULE', 'expression_parens')), [Tree(NonTerminal(Token('RULE', 'expression')), [Tree(NonTerminal(Token('RULE', 'name')), [Token('__ANON_6', 'name')])])])])])]), Token('FROM', 'FROM'), Token('DATA', 'data')])]), Token('EOS', '</s>')])

We can call our parser on this text to get its total weight

In [17]:
g([t.type for t in tokens])

0.0034602076124567475

### Tokenizer State Machines

**TODO**: Dive into the [greenery](https://github.com/qntm/greenery) FSMs to figure out how they work; we can work backward from the Partenon tensor building routine.  Another option might be the [interegular](https://github.com/MegaIng/interegular) library.


**Note**: Tokenizers are FSTs, not FSAs.  However, these libraries implement the kind of restricted FSTs with a separate FSA per token type.

In [18]:
import greenery

def make_greenery_fsms(
    regex_list, 
    ignore = ("\s*",), 
    chars = tuple(chr(i) for i in range(128))
):
    match ignore:
        case []:
            ignore_regex = ""
        case [ignore]:
            ignore_regex = ignore
        case _:
            raise ValueError("ignore must be a list of length at most 1")

    patterns = []
    for regex in regex_list:
        # greenery does not escape spaces
        regex = regex.replace("\\ ", " ")
        patterns.append(greenery.parse(regex + ignore_regex))

    return [pattern.to_fsm() for pattern in patterns]

In [19]:
fsms = make_greenery_fsms([t for t in tokens])

In [20]:
fsms[0]

Fsm(alphabet=frozenset({Charclass((('E', 'E'),)), Charclass((('T', 'T'),)), Charclass((('!', 'B'), ('D', 'D'), ('F', 'K'), ('M', 'R'))), ~Charclass((('\t', 'T'),)), Charclass((('L', 'L'),)), Charclass((('\t', '\r'), (' ', ' '))), Charclass((('\x0e', '\x1f'),)), Charclass((('S', 'S'),)), Charclass((('C', 'C'),))}), states=frozenset({0, 1, 2, 3, 4, 5, 6, 7}), initial=0, finals=frozenset({7}), map={0: {Charclass((('\t', '\r'), (' ', ' '))): 1, Charclass((('\x0e', '\x1f'),)): 1, Charclass((('!', 'B'), ('D', 'D'), ('F', 'K'), ('M', 'R'))): 1, Charclass((('C', 'C'),)): 1, Charclass((('E', 'E'),)): 1, Charclass((('L', 'L'),)): 1, Charclass((('S', 'S'),)): 2, Charclass((('T', 'T'),)): 1, ~Charclass((('\t', 'T'),)): 1}, 1: {Charclass((('\t', '\r'), (' ', ' '))): 1, Charclass((('\x0e', '\x1f'),)): 1, Charclass((('!', 'B'), ('D', 'D'), ('F', 'K'), ('M', 'R'))): 1, Charclass((('C', 'C'),)): 1, Charclass((('E', 'E'),)): 1, Charclass((('L', 'L'),)): 1, Charclass((('S', 'S'),)): 1, Charclass((('T', '

The code below was used in [partenon](https://github.com/probcomp/partenon) to convert the the lark tokenizer `->` greenery FSMs `->` tensors.  We can repurpose it to convert into a transducer from characters to token names.

In [21]:
def make_matrix_from_fsms(fsms, chars):
    max_states = max([len(fsm.states) for fsm in fsms])
    n_inputs = len(chars)

    m_shape = (len(fsms), max_states, n_inputs, max_states)
    m = np.zeros(m_shape, dtype=np.int8)
    finals = jnp.zeros((len(fsms), max_states))
    for fsm_idx, fsm in enumerate(fsms):
        for final_state in fsm.finals:
            finals = finals.at[fsm_idx, final_state].set(1)
        fsm_states = fsm.states
        rejection_states = [e for e in fsm_states if not fsm.islive(e)]
        for state in fsm_states:
            arcs = fsm.map[state]
            for input_char, next_state in arcs.items():
                if next_state in rejection_states:  # rejection state
                    continue
                for char in input_char.get_chars():
                    input_idx = chars.index(char)
                    m[fsm_idx, state, input_idx, next_state] = 1

    m = jnp.array(m)

    return m, finals, max_states

## Prefix Grammar

In [39]:
len(g.cnf.rules), len(g.cnf.prefix_grammar.trim().rules), len(g.cnf.prefix_grammar.trim().rules)/len(g.cnf.rules)

(304, 1220, 4.0131578947368425)

In [44]:
#N = G.nullaryremove()    # could be faster with SCC-based prioritization

In [45]:
g.cnf.prefix_grammar.trim().cnf

0.9999999828532607: #0 → 
0.999999999081103: 99⚡ → 2 3⚡
0.999999999081103: 152⚡ → 2 3⚡
0.0180995475056608: 51⚡ → 5 6⚡
0.07692307687650529: 41⚡ → 5 6⚡
0.01809954746292436: 173⚡ → 5 6⚡
0.018099547432522903: 139⚡ → 5 6⚡
0.06293706287609918: 30⚡ → 5 6⚡
0.018099547505906335: 68⚡ → 5 6⚡
0.01809954746255781: 171⚡ → 5 6⚡
0.07692307687650529: 159⚡ → 5 6⚡
0.01809954750574939: 183⚡ → 5 6⚡
0.06293706287524538: 150⚡ → 5 6⚡
0.018099547432919236: 18⚡ → 5 6⚡
0.06293706287609917: 93⚡ → 5 6⚡
0.06293706279907058: 147⚡ → 5 6⚡
0.01809954746292433: 123⚡ → 5 6⚡
0.06293706287609917: 29⚡ → 5 6⚡
0.018099547432923174: 172⚡ → 5 6⚡
0.018099547505906338: 160⚡ → 5 6⚡
0.018099547489274722: 99⚡ → 5 6⚡
0.018099547505906338: 140⚡ → 5 6⚡
0.07692307691767104: 4⚡ → 5 6⚡
0.018099547489274722: 152⚡ → 5 6⚡
0.018099547505906338: 177⚡ → 5 6⚡
0.06293706287535795: 153⚡ → 5 6⚡
0.018099547462924363: 105⚡ → 5 6⚡
0.0629370628989589: 101⚡ → 5 6⚡
0.018099547505906338: 134⚡ → 5 6⚡
0.018099547505906335: 70⚡ → 5 6⚡
0.07692307691767104: 86⚡ → 5 6⚡
0.018099547505906338: 175⚡ → 5 6⚡
0.018099547479823598: 144⚡ → 5 6⚡
0.018099547505906338: 115⚡ → 5 6⚡
0.018099547462924363: 124⚡ → 5 6⚡
0.018099547462630185: 90⚡ → 5 6⚡
0.06293706279907058: 56⚡ → 5 6⚡
0.018099547479980535: 113⚡ → 5 6⚡
0.018099547505906338: 2⚡ → 5 6⚡
0.01809954747998053: 100⚡ → 5 6⚡
1.0: 7⚡ → 8 2⚡
0.058823529411764705: 160⚡ → 9 10⚡
0.05882352935771194: 99⚡ → 9 10⚡
0.05882352935771194: 152⚡ → 9 10⚡
0.058823529411764705: 134⚡ → 9 10⚡
0.058823529411764705: 175⚡ → 9 10⚡
0.058823529411764705: 2⚡ → 9 10⚡
1.0: 11⚡ → 12 13⚡
0.015384615375893996: 51⚡ → 5 15⚡
0.015384615367869552: 41⚡ → 5 15⚡
0.01538461533956802: 173⚡ → 5 15⚡
0.015384615313726785: 139⚡ → 5 15⚡
0.012587412569139513: 30⚡ → 5 15⚡
0.015384615376102702: 68⚡ → 5 15⚡
0.01538461533925645: 171⚡ → 5 15⚡
0.015384615367869552: 159⚡ → 5 15⚡
0.015384615375969296: 183⚡ → 5 15⚡
0.01258741256896875: 150⚡ → 5 15⚡
0.015384615314063668: 18⚡ → 5 15⚡
0.012587412569139509: 93⚡ → 5 15⚡
0.01258741255373379: 147⚡ → 5 15⚡
0.015384615339567998: 123⚡ → 5 15⚡
0.0999999999919492: 9⚡ → 5 15⚡
0.012587412569139509: 29⚡ → 5 15⚡
0.015384615314067013: 172⚡ → 5 15⚡
0.015384615376102704: 160⚡ → 5 15⚡
0.015384615361965826: 99⚡ → 5 15⚡
0.015384615376102704: 140⚡ → 5 15⚡
0.015384615376102702: 4⚡ → 5 15⚡
0.015384615361965826: 152⚡ → 5 15⚡
0.015384615376102704: 177⚡ → 5 15⚡
0.012587412568991267: 153⚡ → 5 15⚡
0.015384615339568024: 105⚡ → 5 15⚡
0.012587412573711454: 101⚡ → 5 15⚡
0.015384615376102704: 134⚡ → 5 15⚡
0.015384615376102702: 70⚡ → 5 15⚡
0.015384615376102702: 86⚡ → 5 15⚡
0.015384615376102704: 175⚡ → 5 15⚡
0.015384615353932374: 144⚡ → 5 15⚡
0.015384615376102704: 115⚡ → 5 15⚡
0.015384615339568024: 124⚡ → 5 15⚡
0.01538461533931797: 90⚡ → 5 15⚡
0.01258741255373379: 56⚡ → 5 15⚡
0.015384615354065769: 113⚡ → 5 15⚡
0.015384615376102704: 2⚡ → 5 15⚡
0.015384615354065767: 100⚡ → 5 15⚡
0.02450980390164828: #0 → 17 18⚡
1.0: 19⚡ → 20 21⚡
1.0: 22⚡ → 8 12⚡
1.0: 23⚡ → 20 24⚡
0.0180995475056608: 51⚡ → 5 15⚡
0.07692307687650529: 41⚡ → 5 15⚡
0.01809954746292436: 173⚡ → 5 15⚡
0.018099547432522903: 139⚡ → 5 15⚡
0.06293706287609918: 30⚡ → 5 15⚡
0.018099547505906335: 68⚡ → 5 15⚡
0.01809954746255781: 171⚡ → 5 15⚡
0.07692307687650529: 159⚡ → 5 15⚡
0.01809954750574939: 183⚡ → 5 15⚡
0.06293706287524538: 150⚡ → 5 15⚡
0.018099547432919236: 18⚡ → 5 15⚡
0.06293706287609917: 93⚡ → 5 15⚡
0.06293706279907058: 147⚡ → 5 15⚡
0.01809954746292433: 123⚡ → 5 15⚡
0.06293706287609917: 29⚡ → 5 15⚡
0.018099547432923174: 172⚡ → 5 15⚡
0.018099547505906338: 160⚡ → 5 15⚡
0.018099547489274722: 99⚡ → 5 15⚡
0.018099547505906338: 140⚡ → 5 15⚡
0.07692307691767104: 4⚡ → 5 15⚡
0.018099547489274722: 152⚡ → 5 15⚡
0.018099547505906338: 177⚡ → 5 15⚡
0.06293706287535795: 153⚡ → 5 15⚡
0.018099547462924363: 105⚡ → 5 15⚡
0.0629370628989589: 101⚡ → 5 15⚡
0.018099547505906338: 134⚡ → 5 15⚡
0.018099547505906335: 70⚡ → 5 15⚡
0.07692307691767104: 86⚡ → 5 15⚡
0.018099547505906338: 175⚡ → 5 15⚡
0.018099547479823598: 144⚡ → 5 15⚡
0.018099547505906338: 115⚡ → 5 15⚡
0.018099

In [46]:
g.prefix_weight([t.type for t in tokens])

0.0034602076124567475